In [37]:
import os
import re
import sys
import pandas as pd
sys.path.insert(0, './_modules')
from db_functions import TheDB
#from logging_module import getlogger
from logging_module_v2 import StartLogger
from regex_module import FindEmail, FindNumbersBraket, FindLawsuit, FindCaseId
import ast
from sqlalchemy import create_engine
import pymysql

### BD connection

In [2]:
## bots BD connections
conn_dict = {'host':'localhost', 'database': 'brspcqadev',
             'user':'root', 'password':'quito2020'}

cqdb = TheDB(conn_dict)

### Paths and Data:

In [3]:
data_path = os.path.join(os.getcwd(), '_data_output')

In [4]:
gmail_path = os.path.join(os.getcwd(), '_data_original')

In [5]:
df_model = pd.read_csv('model_trix_db.csv', sep = '\t')

In [6]:
df = pd.read_csv(os.path.join(data_path, 'main_trix_clean.csv'), sep = '\t')
print(df.columns)
df = df.rename(columns = {'Case # (DLBCA and Google reference)':'case_id',\
                     'line':'line_id', 'Lawsuit Number':'external_ref_number', 'Deadline':'deadline'})
df = df.loc[:, ['line_id', 'case_id', 'external_ref_number', 'deadline']]
df['external_ref_number'] = df['external_ref_number'].apply(FindLawsuit)
df['case_id'] = df['case_id'].apply(FindCaseId)

Index(['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm'],
      dtype='object')


In [7]:
df

,line_id,case_id,external_ref_number,deadline
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00
3,4,5-6191000002205,,2013-11-22 00:00:00
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00
...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00


In [8]:
## input some date, since those fields are manadatory
df['date_received'] = '2020-08-12'
df['date_received_cq'] = '2020-08-12'

In [9]:
df = df[df['line_id'].str.isdigit() == True]
df['line_id'] = df['line_id'].astype(int)

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,4,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
...,...,...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [11]:
#df['line_id'] = df['line_id'].astype(int) + df_db_trix['line_id'].max()

In [12]:
df = df.drop_duplicates(subset = ['line_id'])

In [13]:
df['date_received'] = pd.to_datetime(df['date_received'])
df['date_received_cq'] = pd.to_datetime(df['date_received_cq'])

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df_test = df[15:].drop('line_id', axis=1)
df_test

,case_id,external_ref_number,deadline,date_received,date_received_cq
15,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12
16,3-8721000000094,0048139-19.2012.8.19.0014,NaN,2020-08-12,2020-08-12
17,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12
18,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12
19,3-4031000002808,0109447-16.2013.8.26.0050,NaN,2020-08-12,2020-08-12
...,...,...,...,...,...
7614,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [15]:
df_model.T

,0
line_id,1
case_id,NaN
external_ref_number,NaN
deadline,0001-01-01 00:00:00
date_received,0001-01-01 00:00:00
date_received_cq,2019-06-17 17:22:28
auth_last_name,NaN
info_case,NaN
case_country,NaN
email_response,NaN


In [16]:
columns_list = list(df_model.columns)

In [17]:
df_clean = pd.DataFrame(columns = columns_list)

In [18]:
df_test

,case_id,external_ref_number,deadline,date_received,date_received_cq
15,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12
16,3-8721000000094,0048139-19.2012.8.19.0014,NaN,2020-08-12,2020-08-12
17,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12
18,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12
19,3-4031000002808,0109447-16.2013.8.26.0050,NaN,2020-08-12,2020-08-12
...,...,...,...,...,...
7614,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [19]:
df_final = pd.concat([df_clean, df_test], sort = False).loc[:, columns_list].fillna('')
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
15,,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
16,,3-8721000000094,0048139-19.2012.8.19.0014,,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
17,,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
18,,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
19,,3-4031000002808,0109447-16.2013.8.26.0050,,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
7615,,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
7616,,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,
7617,,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,,,,,,,,,,


In [20]:
df_final['status'] = 'migração'
df_final['label'] = df_model['label'][0]
df_final['date_created'] = df_model['date_created'][0]
df_final['date_modified'] = df_model['date_modified'][0]
df_final['users_id'] = df_model['users_id'][0]
df_final['user_advice_correction'] = df_model['user_advice_correction'][0]
df_final['advice_approved'] = df_model['advice_approved'][0]
df_final['advice_sent'] = df_model['advice_sent'][0]
df_final['petition_cat_id'] = df_model['petition_cat_id'][0]
df_final['date_status_ms'] = df_model['date_status_ms'][0]
df_final['deadline_ms'] = df_model['deadline_ms'][0]
df_final['user_in_charge_production'] = df_model['user_in_charge_production'][0]
df_final['monitor'] = df_model['monitor'][0]
df_final['emissary_chosen'] = df_model['emissary_chosen'][0]



## fillna
df_final['deadline'].fillna('2020-08-20', inplace = True)

In [21]:
df_final

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
15,,0-9431000002876,0007583-66.2013.403.6109,2014-03-14 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
16,,3-8721000000094,0048139-19.2012.8.19.0014,,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
17,,3-2949000003517,0002607-76.2014.403.6110,2014-05-12 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
18,,8-5005000003432,5013723-77.2014.404.7108,2014-05-14 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
19,,3-4031000002808,0109447-16.2013.8.26.0050,,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7615,,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7616,,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7617,,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,,,,,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,


In [22]:
df_final['deadline'] = df_final['deadline'].replace('', '2020-08-20')
df_final['deadline'] = pd.to_datetime(df_final['deadline'])

In [23]:
df_final = df_final.drop('line_id', axis = 1)

In [24]:
df_final

,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name,info_case,case_country,email_response,status,...,date_status_ms,deadline_ms,ms_info,letter_reviewed,user_in_charge_production,production_completed,production_ready,monitor,emissary_chosen,closure
15,0-9431000002876,0007583-66.2013.403.6109,2014-03-14,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
16,3-8721000000094,0048139-19.2012.8.19.0014,2020-08-20,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
17,3-2949000003517,0002607-76.2014.403.6110,2014-05-12,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
18,8-5005000003432,5013723-77.2014.404.7108,2014-05-14,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
19,3-4031000002808,0109447-16.2013.8.26.0050,2020-08-20,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7615,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7616,0-5628000028599,505-23.2019.4.01.3601,2020-01-03,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,
7617,0-5628000028599,505-23.2019.4.01.3601,2020-12-24,2020-08-12,2020-08-12,,,,,migração,...,2020-08-14 21:21:41,2020-08-14 21:21:41,,,0,,,0,0,


In [25]:
df_final.columns

Index(['case_id', 'external_ref_number', 'deadline', 'date_received',
       'date_received_cq', 'auth_last_name', 'info_case', 'case_country',
       'email_response', 'status', 'targets', 'lers', 'label', 'date_created',
       'date_modified', 'users_id', 'legal_opinion_background',
       'legal_opinion_legal_risk', 'legal_opinion_additional_info',
       'legal_opinion_issue_comments', 'legal_opinion_silo',
       'legal_opinion_petition_comments', 'user_advice_correction',
       'council_review_letter', 'writ_filed_date', 'daily_fines',
       'freezing_assets_date', 'advice_approved', 'advice_pending_yes_no',
       'advice_pendency', 'advice_pendency_solved', 'advice_sent',
       'petition_cat_id', 'petition_yes_no_pending', 'petition_no_explanation',
       'petition_body', 'petition_draft_prepared', 'petition_draft_corrected',
       'petition_field', 'ms_draft_write', 'ms_draft_reviewed', 'ms_filed',
       'status_ms', 'date_status_ms', 'deadline_ms', 'ms_info',
       'l

### The email part:

In [206]:
def GetFirst(input_list, default = ''):
    
    try:
        result = input_list[0]
    except:
        result = default
    
    return result

In [80]:
df_mail = pd.read_csv(os.path.join(gmail_path, 'emails.csv'), sep = '\t', encoding = 'utf-8')
df_mail = df_mail.drop('Unnamed: 0', axis = 1)
df_mail['line_id'] = df_mail['numbers_list'].apply(ast.literal_eval).apply(GetFirst)
df_mail['line_id'] = df_mail['line_id'].astype(int)
df_mail

,path,line_in_path,sender,line_in_mail_filename,trix_counter,numbers_list,numbers_counting,match,line_id
0,/home/leandro/Desktop/Gmail/Linha 9159/_3-8493...,Linha 9159,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9159'],1,['9159'],1,True,9159
1,/home/leandro/Desktop/Gmail/Linha 9692/Re_ RES...,Linha 9692,lis-latam@google.com,['9692'],1,['9692'],1,True,9692
2,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
3,/home/leandro/Desktop/Gmail/Linha 9692/_4-0944...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
4,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
...,...,...,...,...,...,...,...,...,...
19042,/home/leandro/Desktop/Gmail/Linha 9515/_8-7268...,Linha 9515,Helena Costa Rossi,['9515'],1,['9515'],1,True,9515
19043,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10093'],1,['10093'],1,True,10093
19044,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,lis-latam via Google <google@carinaquito.com>,['10093'],1,['10093'],1,True,10093
19045,/home/leandro/Desktop/Gmail/Linha 9741/_4-4742...,Linha 9741,Helena Costa Rossi <helena.rossi@carinaquito.com>,['9741'],1,['9741'],1,True,9741


In [81]:
df_mail = df_mail[df_mail['sender'].str.contains('@google') == False]
df_mail

,path,line_in_path,sender,line_in_mail_filename,trix_counter,numbers_list,numbers_counting,match,line_id
0,/home/leandro/Desktop/Gmail/Linha 9159/_3-8493...,Linha 9159,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9159'],1,['9159'],1,True,9159
2,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
3,/home/leandro/Desktop/Gmail/Linha 9692/_4-0944...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
4,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692
5,/home/leandro/Desktop/Gmail/Linha 10033/_6-513...,Linha 10033,Helena Costa Rossi <helena.rossi@carinaquito.com>,['10033'],1,['10033'],1,True,10033
...,...,...,...,...,...,...,...,...,...
19041,/home/leandro/Desktop/Gmail/Linha 10194/_3-340...,Linha 10194,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10194'],1,['10194'],1,True,10194
19042,/home/leandro/Desktop/Gmail/Linha 9515/_8-7268...,Linha 9515,Helena Costa Rossi,['9515'],1,['9515'],1,True,9515
19043,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10093'],1,['10093'],1,True,10093
19044,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,lis-latam via Google <google@carinaquito.com>,['10093'],1,['10093'],1,True,10093


In [82]:
def MailKiller(input_string):
    final_string = re.sub('<.*?>','', input_string, flags=re.DOTALL)
    return final_string.strip()

In [83]:
df_mail['sender_clean'] = df_mail['sender'].apply(MailKiller)

/home/dev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [105]:
df_mail

,path,line_in_path,sender,line_in_mail_filename,trix_counter,numbers_list,numbers_counting,match,line_id,sender_clean
0,/home/leandro/Desktop/Gmail/Linha 9159/_3-8493...,Linha 9159,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9159'],1,['9159'],1,True,9159,Mohamad Hassan Fares
2,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692,Mohamad Hassan Fares
3,/home/leandro/Desktop/Gmail/Linha 9692/_4-0944...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692,Mohamad Hassan Fares
4,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692,Mohamad Hassan Fares
5,/home/leandro/Desktop/Gmail/Linha 10033/_6-513...,Linha 10033,Helena Costa Rossi <helena.rossi@carinaquito.com>,['10033'],1,['10033'],1,True,10033,Helena Costa Rossi
...,...,...,...,...,...,...,...,...,...,...
19041,/home/leandro/Desktop/Gmail/Linha 10194/_3-340...,Linha 10194,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10194'],1,['10194'],1,True,10194,Rafael Nogaroto Kohl
19042,/home/leandro/Desktop/Gmail/Linha 9515/_8-7268...,Linha 9515,Helena Costa Rossi,['9515'],1,['9515'],1,True,9515,Helena Costa Rossi
19043,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10093'],1,['10093'],1,True,10093,Rafael Nogaroto Kohl
19044,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,lis-latam via Google <google@carinaquito.com>,['10093'],1,['10093'],1,True,10093,lis-latam via Google


In [108]:
adv_list = ['Carina Quito',
'Rafael Nogaroto Kohl',
'Vitor Honofre Bellotto',
'Alice Christina Matsuo',
'Helena Costa Rossi',
'Mohamad Hassan Fares',
'Artur Rovere']



firstname_list = ['Carina',
'Rafael',
'Vitor',
'Alice',
'Helena',
'Mohamad',
'Artur']

In [109]:
df_mail[df_mail['sender'].str.contains('|'.join(firstname_list))]

,path,line_in_path,sender,line_in_mail_filename,trix_counter,numbers_list,numbers_counting,match,line_id,sender_clean
0,/home/leandro/Desktop/Gmail/Linha 9159/_3-8493...,Linha 9159,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9159'],1,['9159'],1,True,9159,Mohamad Hassan Fares
2,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692,Mohamad Hassan Fares
3,/home/leandro/Desktop/Gmail/Linha 9692/_4-0944...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692,Mohamad Hassan Fares
4,/home/leandro/Desktop/Gmail/Linha 9692/RES_ _0...,Linha 9692,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9692'],1,['9692'],1,True,9692,Mohamad Hassan Fares
5,/home/leandro/Desktop/Gmail/Linha 10033/_6-513...,Linha 10033,Helena Costa Rossi <helena.rossi@carinaquito.com>,['10033'],1,['10033'],1,True,10033,Helena Costa Rossi
...,...,...,...,...,...,...,...,...,...,...
19040,/home/leandro/Desktop/Gmail/Linha 9653/_9-7967...,Linha 9653,Mohamad Hassan Fares <mohamad.fares@carinaquit...,['9653'],1,['9653'],1,True,9653,Mohamad Hassan Fares
19041,/home/leandro/Desktop/Gmail/Linha 10194/_3-340...,Linha 10194,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10194'],1,['10194'],1,True,10194,Rafael Nogaroto Kohl
19042,/home/leandro/Desktop/Gmail/Linha 9515/_8-7268...,Linha 9515,Helena Costa Rossi,['9515'],1,['9515'],1,True,9515,Helena Costa Rossi
19043,/home/leandro/Desktop/Gmail/Linha 10093 - Lega...,Linha 10093 - Legal Issue,Rafael Nogaroto Kohl <rafael.kohl@carinaquito....,['10093'],1,['10093'],1,True,10093,Rafael Nogaroto Kohl


In [110]:
df_mail[df_mail['sender'].str.contains('|'.join(firstname_list)) == False]

,path,line_in_path,sender,line_in_mail_filename,trix_counter,numbers_list,numbers_counting,match,line_id,sender_clean
24,/home/leandro/Desktop/Gmail/Linha 9995/FWD_ _1...,Linha 9995,lis-latam via Google <google@carinaquito.com>,['9995'],1,['9995'],1,True,9995,lis-latam via Google
64,/home/leandro/Desktop/Gmail/Linha 9952 - Legal...,Linha 9952 - Legal issue,lis-latam via Google <google@carinaquito.com>,['9952'],1,['9952'],1,True,9952,lis-latam via Google
182,/home/leandro/Desktop/Gmail/Linha 6501 - 7000/...,Linha 6682 - Petition,"""Denise Comim"" <denise.comim@carinaquito.com>",['6682'],1,['6682'],1,True,6682,"""Denise Comim"""
289,/home/leandro/Desktop/Gmail/Linha 6501 - 7000/...,Linha 6772,"""Denise Comim"" <denise.comim@carinaquito.com>",['6772'],1,['6772'],1,True,6772,"""Denise Comim"""
358,/home/leandro/Desktop/Gmail/Linha 6501 - 7000/...,Linha 6847 - Petition,=?UTF-8?Q?=27Ta=C3=ADs_Tesser=27_via_Google?= ...,['6847'],1,['6847'],1,True,6847,=?UTF-8?Q?=27Ta=C3=ADs_Tesser=27_via_Google?=
...,...,...,...,...,...,...,...,...,...,...
18728,/home/leandro/Desktop/Gmail/Linha 5001 - 5500/...,Linha 5016,lis-latam via Google <google@carinaquito.com>,['5016'],1,['5016'],1,True,5016,lis-latam via Google
18732,/home/leandro/Desktop/Gmail/Linha 5001 - 5500/...,Linha 5481,lis-latam via Google <google@carinaquito.com>,['5481'],1,['5481'],1,True,5481,lis-latam via Google
18742,/home/leandro/Desktop/Gmail/Linha 5001 - 5500/...,Linha 5013,lis-latam via Google <google@carinaquito.com>,['5013'],1,['5013'],1,True,5013,lis-latam via Google
18771,/home/leandro/Desktop/Gmail/Linha 5001 - 5500/...,Linha 5023,"""lis-latam via Google"" <google@carinaquito.com>",['5023'],1,['5023'],1,True,5023,"""lis-latam via Google"""


In [111]:
df_grouped = df_mail.groupby('line_id')['sender_clean'].apply(list).apply(set).apply(list).to_frame().reset_index()
df_grouped

,line_id,sender_clean
0,55,[Carina Quito]
1,149,[S & Q Advogados]
2,330,[S & Q Advogados]
3,367,[Alice Christina Matsuo]
4,391,[S & Q Advogados]
...,...,...
8331,10461,[Rafael Nogaroto Kohl]
8332,10462,[Vitor Honofre Bellotto]
8333,10466,[Rafael Nogaroto Kohl]
8334,10467,[Rafael Nogaroto Kohl]


In [113]:
df_grouped['line_id'].min()


55

In [114]:
df_grouped['counting'] = df_grouped['sender_clean'].apply(len)

In [115]:
df_grouped['counting'].value_counts()

1    7781
2     510
3      38
4       7
Name: counting, dtype: int64

In [116]:
df_grouped[df_grouped['counting'] == 3]

,line_id,sender_clean,counting
528,2016,"[Luca Consiglio, Luisa Cassula Piasentini, Err...",3
533,2021,"[Luca Consiglio, Rafael Nogaroto Kohl, Luisa C...",3
914,2427,"[Carina Quito, Vitor Bellotto, Rafael Nogaroto...",3
1353,2885,"[Carina Quito, Rafael Nogaroto Kohl, Luisa Cas...",3
1511,3048,"[Rafael Nogaroto Kohl, Luisa Cassula Piasentin...",3
1541,3079,"[Carina Quito, Rafael Nogaroto Kohl, Luisa Cas...",3
1641,3179,"[Carina Quito, Rafael Nogaroto Kohl, Luisa Cas...",3
2722,4293,"[YouTube Legal Support Team, Carina Quito, Lui...",3
2999,4577,"[, ""Carina Quito"", Rafael Nogaroto Kohl]",3
3361,4959,"[, lis-latam via Google, ""Rafael Kohl""]",3


In [117]:
cqdb.TablesList()

['Access_level',
 'Address',
 'Attachment',
 'Bancos',
 'Delivery_Check',
 'Emissary',
 'Emissary_Status',
 'IP_block',
 'Label',
 'Monitoring',
 'Notifications_log',
 'Rating',
 'Role',
 'Screening',
 'Trix',
 'Trix_1',
 'Users',
 'Users_status',
 'relation_case_ext',
 'relation_case_main',
 'relation_case_thread',
 'relation_ext_case',
 'relation_ext_main',
 'relation_ext_thread',
 'relation_trix_case',
 'relation_trix_ext']

In [156]:
df_users = cqdb.GetTable('Users')

In [157]:
cqdb.TablesDataType('Users')

,0,1,2,3,4,5
0,users_id,int(11),NO,PRI,None,auto_increment
1,first_name,longtext,YES,,None,
2,last_name,longtext,YES,,None,
3,email,longtext,YES,,None,
4,cpf,varchar(45),YES,,None,
5,rg,longtext,YES,,None,
6,oab,longtext,YES,,None,
7,phone,longtext,YES,,None,
8,cellphone,longtext,YES,,None,
9,Password,longtext,YES,,None,


In [158]:
df_users

,users_id,first_name,last_name,email,cpf,rg,oab,phone,cellphone,Password,users_status_id,role_id,level_id,ip_id,adress_id
0,1,Admin,User,admin@suporte.com,000.000.000-00,00.000.000-0,00 000.000,(00) 0000-000,(00) 0 0000-00,8CB2237D0679CA88DB6464EAC60DA96345513964,1,1,1,1,1
1,14,Ana,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
2,15,Joaquina,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32
3,17,Ana2,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
4,18,Joaquina2,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32


In [131]:
df_users2db

,first_name,last_name,Password,users_status_id,role_id,level_id,ip_id,adress_id
0,Carina,Quito,123,1,1,1,1,1


In [167]:
df_users_model = df_users.drop('users_id', axis = 1).loc[1:2,:].copy()
df_users_model.to_csv('users_table_model.csv', sep = '\t', index = False)

In [168]:
df_users_model

,first_name,last_name,email,cpf,rg,oab,phone,cellphone,Password,users_status_id,role_id,level_id,ip_id,adress_id
1,Ana,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
2,Joaquina,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32


In [174]:
df_new = pd.DataFrame(columns = df_users_model.columns)
columns_list = df_new.columns
columns_list

Index(['first_name', 'last_name', 'email', 'cpf', 'rg', 'oab', 'phone',
       'cellphone', 'Password', 'users_status_id', 'role_id', 'level_id',
       'ip_id', 'adress_id'],
      dtype='object')

In [175]:
base_mail = df_users_model['email'][1]
base_cpf = df_users_model['cpf'][1]
base_rg = df_users_model['rg'][1]
base_oab = df_users_model['oab'][1]
base_phone = df_users_model['phone'][1]
base_cellphone = df_users_model['cellphone'][1]
base_password = df_users_model['Password'][1]
base_users_status_id = df_users_model['users_status_id'][1]
base_role_id = df_users_model['role_id'][1]
base_level_id = df_users_model['level_id'][1]
base_ip_id= df_users_model['ip_id'][1]
base_address_id = df_users_model['adress_id'][1]

In [165]:
adv_list

['Carina Quito',
 'Rafael Nogaroto Kohl',
 'Vitor Honofre Bellotto',
 'Alice Christina Matsuo',
 'Helena Costa Rossi',
 'Mohamad Hassan Fares',
 'Artur Rovere']

In [176]:
idx = 0
for fullname in adv_list:
    
    firstname = fullname.split()[0]
    lastname = fullname.split()[-1]
    df_new.at[idx, 'first_name'] = firstname
    df_new.at[idx, 'last_name'] = lastname
    
    df_new.at[idx, 'email'] = base_mail
    df_new.at[idx, 'cpf'] = base_cpf 
    df_new.at[idx, 'rg'] = base_rg
    df_new.at[idx, 'oab'] = base_oab
    df_new.at[idx, 'phone'] = base_phone
    df_new.at[idx, 'cellphone'] = base_cellphone
    df_new.at[idx, 'Password'] = base_password
    df_new.at[idx, 'users_status_id'] = base_users_status_id
    df_new.at[idx, 'role_id'] = base_role_id
    df_new.at[idx, 'level_id'] = base_level_id
    df_new.at[idx, 'ip_id'] = base_ip_id
    df_new.at[idx, 'adress_id'] = base_address_id
    
    
        
        
        
    idx += 1

In [177]:
df_new

,first_name,last_name,email,cpf,rg,oab,phone,cellphone,Password,users_status_id,role_id,level_id,ip_id,adress_id
0,Carina,Quito,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
1,Rafael,Kohl,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
2,Vitor,Bellotto,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
3,Alice,Matsuo,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
4,Helena,Rossi,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
5,Mohamad,Fares,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
6,Artur,Rovere,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30


In [179]:
write_db = False

if write_db == True:
    cqdb.InsertDB(df_new, 'Users', key_column='first_name', return_table=False, append_table=True)

In [180]:
df_users = cqdb.GetTable('Users')

In [181]:
df_users 

,users_id,first_name,last_name,email,cpf,rg,oab,phone,cellphone,Password,users_status_id,role_id,level_id,ip_id,adress_id
0,1,Admin,User,admin@suporte.com,000.000.000-00,00.000.000-0,00 000.000,(00) 0000-000,(00) 0 0000-00,8CB2237D0679CA88DB6464EAC60DA96345513964,1,1,1,1,1
1,14,Ana,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
2,15,Joaquina,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32
3,17,Ana2,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
4,18,Joaquina2,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32
5,19,Carina,Quito,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
6,20,Rafael,Kohl,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
7,21,Vitor,Bellotto,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
8,22,Alice,Matsuo,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
9,23,Helena,Rossi,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30


In [188]:
firstname_list

['Carina', 'Rafael', 'Vitor', 'Alice', 'Helena', 'Mohamad', 'Artur']

In [189]:
df_grouped['counting'].value_counts()

1    7781
2     510
3      38
4       7
Name: counting, dtype: int64

In [190]:
df_grouped[df_grouped['counting']== 2]

,line_id,sender_clean,counting
15,1255,"[Amanda Pilon, S & Q Advogados]",2
21,1373,"[Luca Consiglio, S & Q Advogados]",2
26,1431,"[Andre Rocha Fernandes Pegas, Isabela Gomes de...",2
41,1446,"[Amanda Pilon, Rafael Nogaroto Kohl]",2
53,1462,"[Amanda Pilon, Luisa Cassula Piasentini]",2
...,...,...,...
8197,10274,"[Helena Costa Rossi, Alice Christina Matsuo]",2
8220,10307,"[Helena Costa Rossi, Alice Christina Matsuo]",2
8239,10331,"[Carina Quito, Helena Costa Rossi]",2
8305,10430,"[Carina Quito, Helena Costa Rossi]",2


In [198]:
def MatchNames(input_list):
    return [item for item in input_list if any(name in item for name in firstname_list)]

In [199]:
df_grouped['sender_clean_match'] = df_grouped['sender_clean'].apply(MatchNames)

In [200]:
df_grouped

,line_id,sender_clean,counting,sender_clean_match
0,55,[Carina Quito],1,[Carina Quito]
1,149,[S & Q Advogados],1,[]
2,330,[S & Q Advogados],1,[]
3,367,[Alice Christina Matsuo],1,[Alice Christina Matsuo]
4,391,[S & Q Advogados],1,[]
...,...,...,...,...
8331,10461,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl]
8332,10462,[Vitor Honofre Bellotto],1,[Vitor Honofre Bellotto]
8333,10466,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl]
8334,10467,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl]


In [207]:
df_grouped['fullname'] = df_grouped['sender_clean_match'].apply(GetFirst, default = 'Carina Quito')

In [208]:
df_grouped

,line_id,sender_clean,counting,sender_clean_match,fullname
0,55,[Carina Quito],1,[Carina Quito],Carina Quito
1,149,[S & Q Advogados],1,[],Carina Quito
2,330,[S & Q Advogados],1,[],Carina Quito
3,367,[Alice Christina Matsuo],1,[Alice Christina Matsuo],Alice Christina Matsuo
4,391,[S & Q Advogados],1,[],Carina Quito
...,...,...,...,...,...
8331,10461,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl
8332,10462,[Vitor Honofre Bellotto],1,[Vitor Honofre Bellotto],Vitor Honofre Bellotto
8333,10466,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl
8334,10467,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl


In [210]:
def GetFirstName(input_string):
    return input_string.split()[0]


def GetLastName(input_string):
    return input_string.split()[-1]

In [211]:
df_grouped['first_name'] = df_grouped['fullname'].apply(GetFirstName)

In [212]:
df_grouped['last_name'] = df_grouped['fullname'].apply(GetLastName)

In [213]:
df_grouped

,line_id,sender_clean,counting,sender_clean_match,fullname,first_name,last_name
0,55,[Carina Quito],1,[Carina Quito],Carina Quito,Carina,Quito
1,149,[S & Q Advogados],1,[],Carina Quito,Carina,Quito
2,330,[S & Q Advogados],1,[],Carina Quito,Carina,Quito
3,367,[Alice Christina Matsuo],1,[Alice Christina Matsuo],Alice Christina Matsuo,Alice,Matsuo
4,391,[S & Q Advogados],1,[],Carina Quito,Carina,Quito
...,...,...,...,...,...,...,...
8331,10461,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl,Rafael,Kohl
8332,10462,[Vitor Honofre Bellotto],1,[Vitor Honofre Bellotto],Vitor Honofre Bellotto,Vitor,Bellotto
8333,10466,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl,Rafael,Kohl
8334,10467,[Rafael Nogaroto Kohl],1,[Rafael Nogaroto Kohl],Rafael Nogaroto Kohl,Rafael,Kohl


In [222]:
df_grouped['first_name'].value_counts()

Rafael      1870
Carina      1613
Mohamad     1009
Helena       971
"Helena      920
Vitor        596
"Rafael      542
"Mohamad     524
"Alice       124
Alice         61
"Vitor        58
"Carina       45
Artur          3
Name: first_name, dtype: int64

In [223]:
df_grouped['first_name'] = df_grouped['first_name'].str.replace('"', '')

In [224]:
df_grouped['first_name'].value_counts()

Rafael     2412
Helena     1891
Carina     1658
Mohamad    1533
Vitor       654
Alice       185
Artur         3
Name: first_name, dtype: int64

In [225]:
df_grouped.to_csv('legacy_name_trix_id.csv', sep = '\t', encoding = 'utf-8', index = False)

In [226]:
df_grouped2 = df_grouped.loc[:, ['line_id', 'first_name']]

In [227]:
df_grouped2 

,line_id,first_name
0,55,Carina
1,149,Carina
2,330,Carina
3,367,Alice
4,391,Carina
...,...,...
8331,10461,Rafael
8332,10462,Vitor
8333,10466,Rafael
8334,10467,Rafael


### final names - line_id table

In [230]:
df_legacy = df_users.loc[:, ['first_name', 'users_id']].merge(df_grouped2, on = 'first_name', how = 'right')
df_legacy

,first_name,users_id,line_id
0,Carina,19,55
1,Carina,19,149
2,Carina,19,330
3,Carina,19,391
4,Carina,19,770
...,...,...,...
8331,Mohamad,24,10458
8332,Mohamad,24,10460
8333,Artur,25,9517
8334,Artur,25,10007


In [233]:
df

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,4,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
...,...,...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12


In [236]:
df2 = df.merge(df_legacy, on = 'line_id', how = 'left').drop('first_name', axis = 1)
df2

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12,NaN
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12,NaN
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12,NaN
3,4,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12,NaN
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12,NaN
...,...,...,...,...,...,...,...
7594,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,23.0
7595,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,24.0
7596,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,24.0
7597,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,24.0


In [239]:
df2[df2['users_id'].isna() != True]

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
45,55,1-0230000002988,,2014-04-16 00:00:00,2020-08-12,2020-08-12,19.0
138,149,,43103-32.2013.4.01.3300,NaN,2020-08-12,2020-08-12,19.0
318,330,,,2015-04-14 00:00:00,2020-08-12,2020-08-12,19.0
355,367,,1007672-33.2014.8.26.0008,2015-04-22 00:00:00,2020-08-12,2020-08-12,22.0
379,391,,4614-05.2014.4.01.4103,2015-05-22 00:00:00,2020-08-12,2020-08-12,19.0
...,...,...,...,...,...,...,...
7594,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,23.0
7595,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,24.0
7596,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,24.0
7597,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,24.0


In [240]:
df_users

,users_id,first_name,last_name,email,cpf,rg,oab,phone,cellphone,Password,users_status_id,role_id,level_id,ip_id,adress_id
0,1,Admin,User,admin@suporte.com,000.000.000-00,00.000.000-0,00 000.000,(00) 0000-000,(00) 0 0000-00,8CB2237D0679CA88DB6464EAC60DA96345513964,1,1,1,1,1
1,14,Ana,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
2,15,Joaquina,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32
3,17,Ana2,Paula,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
4,18,Joaquina2,Trix,joaquinatrix@carinaquito.com,123.545.845-58,35.484.566-4,123547899999999999999,(11) 2 2255-4588,(85) 4 8455-5554,8CB2237D0679CA88DB6464EAC60DA96345513964,1,2,2,1,32
5,19,Carina,Quito,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
6,20,Rafael,Kohl,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
7,21,Vitor,Bellotto,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
8,22,Alice,Matsuo,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30
9,23,Helena,Rossi,paula.aragao@carinaquito.com,322.895.558-35,11.111.111-1,000.000,(11) 9843-3182,(11) 8 8888-8888,CEA8BE18F8249FDBAAA535B000505661DD160000,1,1,1,1,30


In [244]:
df2['users_id'] = df2['users_id'].fillna(19).astype(int)

In [245]:
df2

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,users_id
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12,19
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12,19
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12,19
3,4,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12,19
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12,19
...,...,...,...,...,...,...,...
7594,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00,2020-08-12,2020-08-12,23
7595,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00,2020-08-12,2020-08-12,24
7596,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00,2020-08-12,2020-08-12,24
7597,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00,2020-08-12,2020-08-12,24


In [246]:
df2['users_id'].value_counts()

19    3408
20    1817
23    1027
24     617
21     598
22     132
Name: users_id, dtype: int64

In [26]:

send2DB = False

if send2DB == True:


    ## create sqlalchemy engine
    engine = create_engine("mysql://{user}:{pw}@{host}/{db}?charset=utf8mb4"
                           .format(user= conn_dict['user'],
                                   pw=conn_dict['password'],
                                   host = conn_dict['host'],
                                   db=conn_dict['database']), encoding="utf8")


    df_final.to_sql('Trix', con = engine, if_exists = 'append',\
                index = False, chunksize = 1000)